<H1>DRAW SWARM ANALYSIS</H1>

In [2]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 120)

<h2>LOAD DATA</h2>

In [3]:
S=pd.read_csv('./data/csv/exp.p.csv')
C=pd.read_csv('./data/csv/exp.c.csv')
R=pd.read_csv('./data/csv/exp.r.csv')

<h2>EXTRACT STEP</h2>
Set <code>step</code> for position in simulation<br>
Set <code>id</code> for individual agent of interest.

In [10]:
step=1
id=23
#SWARM DATA
AGENTS=S[["STEP","ID","X","Y","PERIM"]].query("STEP==%d" % step)
COHESION=C[["STEP","PID","PX","PY","NID","NX","NY","PPERIM","NPERIM"]].query("STEP==%d" % step)
COHESIONP=COHESION.query("PPERIM==True and NPERIM==True")
REPULSION=R[["STEP","PID","PX","PY","NID","NX","NY",]].query("STEP==%d" % step)

#AGENT DATA
AGENT=AGENTS.query("ID==%d" % id)
ACOHESION=COHESION.query("PID==%d" % id)
ACOHESIONP=ACOHESION.query("PPERIM==True and NPERIM==True")
AREPULSION=REPULSION.query("PID==%d" % id)


In [11]:
#NEIGHBOURS LINK DATA
NEIGHBOURS=ACOHESION["NID"]
#NEIGHBOUR LINKS FOR STEP EXCLUDING THE TARGET AGENT 
STEPCOH=COHESION.query("STEP==%d and PID!=%d and NID!=%d" % (step,id,id))
# FILTER NEIGHBOUR STARTING POINTS USING A LIST
STARTCOH=STEPCOH[STEPCOH['PID'].isin(NEIGHBOURS)]
# FILTER NEIGHBOUR END POINTS USING A LIST
NEIGHBOURCOH=STARTCOH[STARTCOH['NID'].isin(NEIGHBOURS)]

<h2>SWARM GRAPH</h2><br>
FULL SWARM GRAPH

In [12]:
%matplotlib qt
plt.scatter(AGENTS["X"],AGENTS["Y"],s=100,color="green")
for i,row in COHESION.iterrows():
    plt.annotate("%5d"%row["PID"],(row["PX"],row["PY"]))
for i,row in COHESION.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="lightgreen")
for i,row in COHESIONP.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="darkgreen")

plt.title("Swarm Cohesion at STEP:[%d]" % step)
plt.grid()
plt.gca().set_aspect("equal")
plt.xlabel("X")
plt.ylabel("Y")
plt.tight_layout()
plt.show()

REPULSION GRAPH

In [37]:
%matplotlib qt
for i,row in REPULSION.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],"r-")
plt.scatter(AGENTS["X"],AGENTS["Y"],s=100)
plt.title("Swarm Repulsion at STEP:[%d]" % step)
plt.grid()
plt.gca().set_aspect("equal")
plt.xlabel("X")
plt.ylabel("Y")
plt.tight_layout()

<H3>INDIVIDUAL AGENT GRAPHS</H3><br>
AGENT AND NEIGHBOURS

In [38]:
%matplotlib qt
plt.scatter(AGENT["X"],AGENT["Y"],s=100,color="green")
plt.scatter(ACOHESION["NX"],ACOHESION["NY"],s=100,color="green")
for i,row in ACOHESION.iterrows():
    plt.annotate("%5d"%row["PID"],(row["PX"],row["PY"]))
for i,row in ACOHESION.iterrows():
    plt.annotate("%5d"%row["NID"],(row["NX"],row["NY"]))
for i,row in ACOHESION.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="lightgreen")
for i,row in ACOHESIONP.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="darkgreen")

plt.title("Single Agent [%d] Swarm Cohesion at STEP:[%d]" % (id,step))
plt.grid()
plt.gca().set_aspect("equal")
plt.xlabel("X")
plt.ylabel("Y")
plt.tight_layout()
plt.show()

In [ ]:
ACOHESION.to_csv("agent.csv")

AGENT AND NEIGHBOURS WITH NEIGHBOUR LINKS SHOWN

In [8]:
%matplotlib qt
plt.scatter(AGENT["X"],AGENT["Y"],s=100,color="green")
plt.scatter(ACOHESION["NX"],ACOHESION["NY"],s=100,color="green")
for i,row in NEIGHBOURCOH.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="black")
for i,row in ACOHESION.iterrows():
    plt.annotate("%5d"%row["PID"],(row["PX"],row["PY"]))
for i,row in ACOHESION.iterrows():
    plt.annotate("%5d"%row["NID"],(row["NX"],row["NY"]))
for i,row in ACOHESION.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="lightgreen")
for i,row in ACOHESIONP.iterrows():
    plt.plot([row["PX"], row["NX"]], [row["PY"], row["NY"]],color="darkgreen")

plt.title("Single Agent [%d] Swarm Cohesion at STEP:[%d]" % (id,step))
plt.suptitle("Showing neighbour links")
plt.grid()
plt.gca().set_aspect("equal")
plt.xlabel("X")
plt.ylabel("Y")
plt.tight_layout()
plt.show()

<H3>SWARM ANIMATION</H3>

In [6]:
%matplotlib qt
from matplotlib import animation
from matplotlib.animation import FuncAnimation

loop = S.STEP.max()
fig = plt.figure()
xLimMin = S.X.min()
xLimMax = S.X.max()
yLimMin = S.Y.min()
yLimMax = S.Y.max()
plt.xlim([xLimMin,xLimMax])
plt.ylim([yLimMin,yLimMax])
plt.grid()
plt.gca().set_aspect("equal")
PLOT=S[["STEP","ID","X","Y"]].query("STEP==%d" % 1)
COHESION=C[["STEP","PX","PY","NX","NY",]].query("STEP==%d" % 1)
COHESIONP=C[["STEP","PX","PY","NX","NY","PPERIM","NPERIM"]].query("PPERIM==True and NPERIM==True and STEP==%d" % 1)
scat = plt.scatter(PLOT["X"],PLOT["Y"],s=10,color="green")
plt.title('Time (s): 1')

def animate(i):
    PLOT=S[["STEP","ID","X","Y"]].query("STEP==%d" % i)
    scat.set_offsets(np.c_[PLOT["X"],PLOT["Y"]])
    plt.title('Time (s): %d or %d' % (i+1,loop))

anim = FuncAnimation(fig, animate, interval=15, frames=loop, repeat=True)
plt.draw() 

